# Loading and Adjusting Sharadar US Equity Prices

The notebook demonstrates and tests the get_pricing method for loading and adjusting the Sharadar SEP bundle (ingested with the sep_ingest notebook in this repository). The corresponding example consists of five steps: (1) retrieving adjusted data with get_pricing from Zipline, (2) reading split-adjusted data from Sharadar files, (3) testing unadjusted Sharadar data, (4) testing the split and dividend adjustments, and (5) testing split- and dividend-adjusted data generated by the get_pricing method.

In [1]:
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

Importing the fsharadar.sep module from the Flounder extension

In [2]:
from fsharadar import sep

Getting access to the SEP bundle via three classes: AssetFinder, DailyBcolzReader, and SQLiteAdjustmentReader.
In this notebook,  they will be used for getting a list of assets (Section 1) and testing intermediate results, 
such as Zipline unadjusted prices (Section 3) and  adjustments (Section 4). 

In [3]:
sep_bundle_data = sep.load()

In [4]:
sep_asset_finder = sep_bundle_data.asset_finder
pricing_reader = sep_bundle_data.equity_daily_bar_reader
adjustment_reader = sep_bundle_data.adjustment_reader

## 1. Getting Historical Prices Adjusted for Splits and Dividends

In [5]:
# 1. Selecting the time interval 

start_date = "2011-01-03"; end_date = "2021-02-12"

In [6]:
# 2. Selecting one field from Sharadar SEP bundle

field = 'close'
sep.bundle_tags

['close', 'high', 'low', 'open', 'volume']

In [7]:
# 3. Getting all tickers maintained in Sharadar SEP bundle

sep_asset_finder = sep_bundle_data.asset_finder
sep_assets = sep_asset_finder.retrieve_all(sep_asset_finder.sids)
len(sep_assets)

19426

Applying get_pricing for loading adjusted stock prices

In [8]:
%%time

prices = sep.get_pricing(sep_assets, start_date, end_date, field)
prices.dropna(axis=1, how='all', inplace=True)

CPU times: user 7.4 s, sys: 1.54 s, total: 8.93 s
Wall time: 7.48 s


In [9]:
prices.head(2)

,Equity(101286 [TUXS]),Equity(101386 [GBBT]),Equity(101501 [BBUCQ]),Equity(101512 [GOVB]),Equity(101923 [CIBN]),Equity(103609 [AMCRY]),Equity(103628 [AXAHY]),Equity(103638 [NHYDY]),Equity(103642 [KKPNY]),Equity(103694 [LFRGY]),...,Equity(635425 [MOLXA]),Equity(635463 [TFONY]),Equity(635469 [SPCHA]),Equity(635482 [BNHNA]),Equity(635494 [GROM]),Equity(635639 [TDSS]),Equity(635664 [CADTR]),Equity(635677 [SPCE.U]),Equity(635686 [ABV]),Equity(635746 [QUBT])
2011-01-03 00:00:00+00:00,1.08,NaN,0.01,6.787,4.063,15.488,10.385,5.364,5.033,16.05,...,19.23,16.32,2.93,8.00,NaN,31.71,NaN,NaN,6.358,NaN
2011-01-04 00:00:00+00:00,1.08,NaN,0.01,6.787,4.063,15.341,10.574,5.264,5.053,16.17,...,19.20,16.50,2.82,8.04,NaN,31.86,NaN,NaN,6.242,NaN


In [10]:
assets = prices.columns
symbols = [asset.symbol for asset in assets]

## 2. Reading Sharadar Data for Testing Zipline Unadjusted and Adjusted Prices

In [11]:
sharadar_dir = './'

Tickers (with sids)

In [12]:
sharadar_tickers_file = sharadar_dir + "SHARADAR_TICKERS.csv"

In [13]:
tickers_df = pd.read_csv(sharadar_tickers_file)
print(len(tickers_df.ticker.unique()))

24992


SEP (Sharadar Equity Prices) 

In [14]:
sharadar_sep_file = sharadar_dir + "SHARADAR_SEP.csv"

In [15]:
%%time
from fsharadar.read import read_data_file

sep_df = read_data_file(sharadar_sep_file, tickers_df, sep.bundle_tags)
print(len(sep_df.symbol.unique()))

19442
CPU times: user 43.7 s, sys: 2.27 s, total: 46 s
Wall time: 47.1 s


In [16]:
sep_df = sep_df[sep_df.symbol.isin(symbols)]
print(len(sep_df.symbol.unique()))

11399


Actions (dividends, splits, etc.)

In [17]:
actions_file = sharadar_dir + "SHARADAR_ACTIONS.csv"

In [18]:
%%time

from fsharadar.read import read_actions_file

# read actions_file (with splits)
actions_df = read_actions_file(actions_file, tickers_df)

CPU times: user 571 ms, sys: 7.5 ms, total: 579 ms
Wall time: 590 ms


In [19]:
actions_df = actions_df[actions_df.symbol.isin(symbols)]
print(len(actions_df.symbol.unique()))

11399


## 3. Testing  Unadjusted Prices

In [20]:
pricing_reader = sep_bundle_data.equity_daily_bar_reader
calendar = pricing_reader.trading_calendar

### 3.1 Unadjusting Sharadar Data

In [21]:
%%time

from fsharadar.read import unadjust_splits

unadj_sep_df, t = unadjust_splits(sep_df, actions_df, calendar)

100%|██████████| 11399/11399 [02:30<00:00, 75.84it/s]


CPU times: user 2min 58s, sys: 3.09 s, total: 3min 1s
Wall time: 3min


### 3.2 Comparing Zipline and Sharadar Unadjusted Prices

In [22]:
%%time

unadj_sep_df = unadj_sep_df.set_index(['symbol', 'date'])
unadj_sep_df.sort_index(level=0, inplace=True)

CPU times: user 19 s, sys: 1.32 s, total: 20.3 s
Wall time: 20.3 s


In [23]:
from tqdm import tqdm

for tag in sep.bundle_tags:
    
    raw_array, = pricing_reader.load_raw_arrays([tag], prices.index[0], prices.index[-1], assets,)  
    
    for i in tqdm(range(0, len(assets)), desc='Processing {}'.format(tag)):
    
        asset_values = raw_array[:, i]
        asset_values = asset_values[~np.isnan(asset_values)]

        unadj_sep_xs = unadj_sep_df.xs(symbols[i]).loc[start_date: end_date][tag].dropna()  
    
        if len(asset_values) != len(unadj_sep_xs):
            print(i, symbols[i], 'different sizes:', len(asset_values), len(unadj_sep_xs))
            continue
    
        std_diff = np.std(asset_values - unadj_sep_xs.values)
        if std_diff > 1.e-9: print(symbols[i], std_diff)

Processing volume: 100%|██████████| 11399/11399 [00:20<00:00, 557.22it/s]


## 4. Testing Adjustments

In [24]:
adjustment_reader = sep_bundle_data.adjustment_reader

### 4.1 Split Adjustments

In [25]:
%%time

actions_df = actions_df.set_index(['symbol', 'date'])
actions_df.sort_index(level=0, inplace=True)

CPU times: user 188 ms, sys: 99 µs, total: 188 ms
Wall time: 187 ms


In [26]:
from tqdm import tqdm

table_name = 'SPLITS'

for i in tqdm(range(0, len(assets))):
    
    sid = assets[i].sid
    symbol = assets[i].symbol

    split_adjustments = adjustment_reader.get_adjustments_for_sid(table_name, sid)
    
    if len(split_adjustments)  == 0:
        continue
        
    actions_xs = actions_df.xs(symbol)
    
    split_ratios = actions_xs[actions_xs.action == 'split'].value
    split_ratios = 1./split_ratios
 
    split_ratios = split_ratios.sort_index(ascending=False)
    split_ratios = split_ratios.tz_localize('UTC')
 
    if not split_ratios.index.equals(pd.to_datetime(np.array(split_adjustments)[:, 0])):
        print(i, symbol, 'dates are different')
    
    if not np.array_equal(split_ratios.values, np.array(split_adjustments)[:, 1]):
        print(i, symbol, 'values are different')

100%|██████████| 11399/11399 [00:04<00:00, 2397.28it/s]


### 4.2 Dividend Adjustments

In [27]:
def calc_dividend_ratios(unadj_xs):
    
    unadj_xs = unadj_xs.sort_index(ascending=False)
    
    closes = unadj_xs['close']
    dividends = unadj_xs['dividends'].shift(1)
    dividends[0] = 0.0
    
    dividend_ratios = (1. - dividends/closes)
    
    return dividend_ratios

In [28]:
from tqdm import tqdm

table_name = 'DIVIDENDS'

for i in tqdm(range(0, len(assets))):
    
    sid = assets[i].sid
    symbol = assets[i].symbol

    dividend_adjustments = adjustment_reader.get_adjustments_for_sid(table_name, sid)
    
    if len(dividend_adjustments)  == 0:
        continue
        
    unadj_sep_xs = unadj_sep_df.xs(symbol)
    dividend_ratios = calc_dividend_ratios(unadj_sep_xs)
    
    dividend_ratios = dividend_ratios.shift(-1)[:-1]
    dividend_ratios = dividend_ratios[dividend_ratios != 1.0]
    
    dividend_ratios = dividend_ratios.sort_index(ascending=True)
    dividend_ratios = dividend_ratios.tz_localize('UTC')
    
    if not dividend_ratios.index.equals(pd.to_datetime(np.array(dividend_adjustments)[:, 0])):
        print(i, symbol, 'dates are different')

    if not np.array_equal(dividend_ratios.values, np.array(dividend_adjustments)[:, 1]):
        print(i, symbol, 'values are different')

100%|██████████| 11399/11399 [00:17<00:00, 658.47it/s] 


## 5. Testing Adjusted Prices

In [29]:
# Some splits are provided outside of the business calendar
# therefore date_range is used

date_range = pd.date_range(start=start_date, end=end_date)
date_range_df = pd.DataFrame(index=date_range)
date_range_df = date_range_df.sort_index(ascending=False)

In [30]:
def calc_split_ratios(actions_xs):

    splits = actions_xs[actions_xs.action == 'split'].value
    splits = 1./splits

    # Descending order is inherited from date_range
    splits = date_range_df.join(splits).value
    splits = splits.replace(np.nan, 1.0)
    
    splits = splits.shift(1)
    splits[0] = 1.0
    
    return splits

In [31]:
from tqdm import tqdm

for i in tqdm(range(0, len(assets))):
    
    asset = assets[i]
    sid = asset.sid
    symbol = asset.symbol
    
    unadj_sep_xs = unadj_sep_df.xs(symbol).loc[start_date: end_date]
    unadj_sep_xs = unadj_sep_xs.sort_index(ascending=False)
    
    adj_prices = unadj_sep_xs[field].dropna()
    
    split_adjustments = adjustment_reader.get_adjustments_for_sid('SPLITS', sid)
    if len(split_adjustments)  != 0:
        actions_xs = actions_df.xs(symbol).loc[start_date: end_date]
        split_ratios = calc_split_ratios(actions_xs)
        splits_adjustments = split_ratios.cumprod()
        split_adjustments = unadj_sep_xs.join(splits_adjustments).value # Switching to the business dates
        adj_prices = (adj_prices*split_adjustments).dropna()
    
    dividend_adjustments = adjustment_reader.get_adjustments_for_sid('DIVIDENDS', sid)
    if len(dividend_adjustments)  != 0:
        dividend_ratios = calc_dividend_ratios(unadj_sep_xs)
        dividend_adjustments = dividend_ratios.cumprod()
        adj_prices = (adj_prices*dividend_adjustments).dropna()
        
    adj_prices.sort_index(ascending=True, inplace=True)    
    
    asset_prices_ts = prices[asset].dropna() 
    std_diff = np.std(adj_prices.values - asset_prices_ts.values)
    
    if std_diff > 5.e-4:
        print(i, symbol, std_diff)  


100%|██████████| 11399/11399 [00:50<00:00, 223.91it/s]
